# Downloading and read data

First, download the data from [Kaggle](https://www.kaggle.com/datasets/tirendazacademy/fifa-world-cup-2022-tweets/) to the path "../data/fifa-tweet-sentiments". Then, extract the .csv in that same directory.

In [8]:
import pandas as pd

data_path = "../data/fifa-tweet-sentiments/fifa_world_cup_2022_tweets.csv"
df = pd.read_csv(data_path)
df.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment
0,0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadB...,neutral
1,1,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,positive
2,2,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 htt...,positive
3,3,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/...,positive
4,4,2022-11-20 23:58:28+00:00,0,Twitter for Android,"If the BBC cares so much about human rights, h...",negative


Make train/test splits

In [9]:
sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2}
pre_df = pd.DataFrame({'text': df['Tweet'], 'label': df['Sentiment'].apply(lambda x: sentiment_map[x])})

In [10]:
from sklearn.model_selection import train_test_split

train_split, test_split = train_test_split(pre_df, train_size=0.7)
train_split.shape

(15766, 2)

## Finetuning a pretrained huggingface sentiment classifier

Most of the following is adapted from section 3.a of "[Getting Started with Sentiment Analysis using Python](https://huggingface.co/blog/sentiment-analysis-python)." This workflow loads a pre-trained DistilBERT model and finetunes a classifier head on top of it. We take this approach to classify tweets from the FIFA dataset above. 

Get pretrained tokenizer and collator to transform input and pad input text.

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Define helper function for calculating accuracy.

In [13]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    return{"accuracy": accuracy}

Authenticate the notebook so we can write to our repo. Make sure to use a token with `write` privileges.

In [7]:
from huggingface_hub import notebook_login
notebook_login() # login with a write token here

Get a pretrained model. Note that we use 3 labels since our data has three possible sentiments (`negative`, `neutral`, `positive`).

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map the string values for sentiment values to ints, and make a Dataset-compatible dataframe for the huggingface trainer.

Make the DatasetDict object for training/eval.

In [10]:
import datasets

train_dataset = datasets.Dataset.from_dict(train_split)
test_dataset = datasets.Dataset.from_dict(test_split)

In [11]:
def preprocess(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset.map(preprocess, batched=True)
tokenized_test = test_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/15766 [00:00<?, ? examples/s]

Map:   0%|          | 0/6758 [00:00<?, ? examples/s]

Train the model!

In [12]:
from transformers import TrainingArguments, Trainer

repo_name = f"finetuning-sentiment-model-fifa-{train_split.shape[0]}-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=1e-2,
    save_strategy="epoch",
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Evaluate the model on the eval set.

In [ ]:
trainer.evaluate()

{'eval_loss': 0.711454451084137,
 'eval_accuracy': 0.8393015685113939,
 'eval_runtime': 7.1526,
 'eval_samples_per_second': 944.835,
 'eval_steps_per_second': 59.14,
 'epoch': 5.0}

Push the model to the remote repo, and evaluate the trained model on some unseen tweets.

In [ ]:
trainer.push_to_hub()

In [8]:
from transformers import pipeline

back_map = {val: key for key, val in sentiment_map.items()}
huggingface_username = "<YOUR_HUGGINGFACE_USERNAME>"

sentiment_model = pipeline(model=f"{huggingface_username}/finetuning-sentiment-model-fifa-{train_split.shape[0]}-samples")
examples = ["This team is the greatest!", "I can't with this team...", "This game is a snoozefest", "What is soccer?"]
preds = sentiment_model(examples)
for i, pred in enumerate(preds):
    print(f'{back_map[int(pred["label"][-1])]} with score={pred["score"]:6.5f} ("{examples[i]}")')

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

positive with score=0.99932 ("This team is the greatest!")
negative with score=0.99918 ("I can't with this team...")
negative with score=0.99802 ("This game is a snoozefest")
neutral with score=0.99411 ("What is soccer?")


## Sentiment classification using GPT-3.5

How well can a finetuned GPT-3.5 do sentiment classification on this dataset? To answer this, we'll finetune the model on a few prompt/completions pairs.

First, we define a few helper methods to help us train (finetune) and evaluate the LLM.

In [66]:
import json

def makeFinetuneData(split, path, N, n_batch, bs):
    with open(path, 'w') as outfile:
        for i in range(n_batch):
            messages = messagesFromSplit(split, i*bs, (i+1)*bs)

            json.dump({'messages': messages}, outfile)
            outfile.write('\n')

def messagesFromSplit(split, i_s, i_e, include_assistant=True):
    # format the system/user/assistant content

    messages = []

    # system message
    messages.append({'role': 'system', 'content': "Classify each tweet in the batch as 0 (negative), 1 (neutral), or 2 (positive)."})

    # user message
    messages.append({'role': 'user', 'content': ", ".join([f"<TWEET>{[split.iat[j,0]]}</TWEET>" for j in range(i_s, i_e)])})

    if include_assistant:
        # assistant message
        messages.append({'role': 'assistant', 'content': ", ".join([f"{split.iat[j,1]}" for j in range(i_s, i_e)])})

    return messages

def calcAccuracy(df):
    acc = 0
    N = df.shape[0]
    for i in range(df.shape[0]):
        acc += 1 / N if df.at[i,'label'] == df.at[i,'prediction'] else 0
    return acc

Make training data for finetuning.

In [14]:
import json

# same dir as the kaggle data
train_path = "../data/fifa-tweet-sentiments/fifa_world_cup_2022_train_prompts.jsonl"
out = {}
N = 80
n_batch = 10
bs = N // n_batch

makeFinetuneData(train_split, train_path, N, n_batch, bs)


In [2]:
OPENAI_API_KEY="<YOUR_OPENAI_API_KEY>"

In [15]:
# upload the training file
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

file_obj = client.files.create(
  file=open(train_path, "rb"),
  purpose="fine-tune"
)

FileObject(id='file-KmXUOHSN0mI9lb9oH9VoAE0N', bytes=17176, created_at=1703260768, filename='fifa_world_cup_2022_train_prompts.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [20]:
# kick-off a training job
client = OpenAI(api_key=OPENAI_API_KEY)
client.fine_tuning.jobs.create(
  training_file=file_obj.id, 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-XOrIG6yeh64kuKAxrpSAAkXW', created_at=1703260988, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-z274jjuiMMCtIYX9lKyIFDD8', result_files=[], status='validating_files', trained_tokens=None, training_file='file-KmXUOHSN0mI9lb9oH9VoAE0N', validation_file=None)

In [22]:
# List your current fine-tuning jobs
jobs = client.fine_tuning.jobs.list(limit=10)
jobs

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-XOrIG6yeh64kuKAxrpSAAkXW', created_at=1703260988, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-z274jjuiMMCtIYX9lKyIFDD8', result_files=[], status='running', trained_tokens=None, training_file='file-KmXUOHSN0mI9lb9oH9VoAE0N', validation_file=None)], object='list', has_more=False)

In [29]:
# Retrieve the state of your fine_tuning job 
state = client.fine_tuning.jobs.retrieve(jobs.data[0].id)
state

FineTuningJob(id='ftjob-XOrIG6yeh64kuKAxrpSAAkXW', created_at=1703260988, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:personal::8YcO68pw', finished_at=1703261405, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-z274jjuiMMCtIYX9lKyIFDD8', result_files=['file-5OzY6g0bAbPsE5GXWrHqCqTk'], status='succeeded', trained_tokens=55120, training_file='file-KmXUOHSN0mI9lb9oH9VoAE0N', validation_file=None)

In [31]:
# get the model name
ft_model_name = state.fine_tuned_model

With the fine-tuned model available, we now call this model on a few prompts from the test set.

In [62]:
from tqdm import trange

batch_size = 8
N = 10

def callSentimentClfLllm(client, split, N, bs, model_name):

    for i in trange(N):
        messages = messagesFromSplit(split, i*bs, (i+1)*bs, include_assistant=False)

        # print(prompts)
        response = client.chat.completions.create(
            model = model_name,
            messages=messages
        )

        for j, val in enumerate(response.choices[0].message.content.split(", ")):
            row = split.iloc[i*bs+j,:]
            df_gpt = pd.concat([pd.DataFrame([[int(val),
                                            row['label'],
                                            row['text']]], columns=df_gpt.columns), df_gpt], ignore_index=True)
    
    return df_gpt

df_gpt = callSentimentClfLllm(client, test_split, N, batch_size, ft_model_name)

100%|██████████| 10/10 [00:06<00:00,  1.57it/s]


In [63]:
calcAccuracy(df_gpt)

0.6999999999999995


On this small sample from the test split, we've achieved about 70% accuracy. Let's see the performance on the larger test split.

In [65]:
N = test_split.shape[0] // batch_size
df_gpt = callSentimentClfLllm(client, test_split, N, batch_size)

100%|██████████| 844/844 [08:52<00:00,  1.58it/s]


In [67]:
calcAccuracy(df_gpt)

0.7270170244263887

So finetuning has helped improve the GPT-3 based classifier to about 72.7% test accuracy. This is still not quite at the level of the huggingface classifier (83.9% test accuracy), and we could potentially iterate over our finetuning/prompting process to improve the LLM's accuracy. This could quickly get expensive, so we'll save that as an exercise for another time!